### The executor

Now that we have our `DistOptController`, it's time to take care of the actual execution of the algorithm at the client level - we'll build on top of the NVFlare `Executor` to do to that.

In NVFlare, an `Executor` is a client-side component that handles tasks received from the controller and executes them. For our purposes we'll need our executor to be able to do a few things:
- receive the config from the server/controller
- communicate with its neighbors and send/receive messages to/from them
- run the algorithm

For the moment, we'll focus on synchronous algorithms only, meaning that the clients will need to run the iterations of an algorithm in a synchronous way. Let' call our executor `SyncAlgorithmExecutor`.
The only method that must be implemented in this case is the `execute` method, which takes the `task_name` and `shareable` sent from the controller as inputs.

```python
from nvflare.apis.executor import Executor
from nvflare.apis.fl_constant import ReturnCode
from nvflare.apis.fl_context import FLContext
from nvflare.apis.shareable import Shareable, make_reply


class SyncAlgorithmExecutor(Executor):

    def execute(
        self,
        task_name: str,
        shareable: Shareable,
        fl_ctx: FLContext,
        abort_signal: Signal,
    ):
        if task_name == "config":
            # TODO: receive and store config
            ...
            return make_reply(ReturnCode.OK)

        elif task_name == "run_algorithm":
            # TODO: run the algorithm
            return make_reply(ReturnCode.OK)
        else:
            self.log_warning(fl_ctx, f"Unknown task name: {task_name}")
            return make_reply(ReturnCode.TASK_UNKNOWN)
```

Let's focus on the execution of the `"config"` task - for this we can just create some attributes to store the config and the neighbors (and the local weight computed from them).

```python
from nvflare.apis.dxo import from_shareable
from nvdo.types import LocalConfig, Neighbor


class SyncAlgorithmExecutor(Executor):
    def __init__(self):
        super().__init__()

        self.config = None
        self._weight = None
        self.neighbors: list[Neighbor] = []


    def execute(
        self,
        task_name: str,
        shareable: Shareable,
        fl_ctx: FLContext,
        abort_signal: Signal,
    ):
        if task_name == "config":
            # Receive and store config
            self.config = LocalConfig(**from_shareable(shareable).data)
            self.neighbors = self.config.neighbors
            self._weight = 1.0 - sum([n.weight for n in self.neighbors])
            return make_reply(ReturnCode.OK)

        elif task_name == "run_algorithm":
            # TODO: run the algorithm
            return make_reply(ReturnCode.OK)
        else:
            self.log_warning(fl_ctx, f"Unknown task name: {task_name}")
            return make_reply(ReturnCode.TASK_UNKNOWN)
```

That was relatively easy - so, now to the slightly more challenging part of letting clients communicate with each other.
To do that we'll do a few things:
- we'll use the `send_aux_request` method to let a client send a message to its neighbors
- we'll need to register a callback to handle received messages (via the `register_aux_message_handler` function) and add an attribute `neighbors_values` to store received values. We'll call the callback `_handle_neighbor_value` and the registration will be done in the `handle_event` method at start time (i.e., when receiving the `EventType.START_RUN` event). Other events can be handled in the same way if needed.
- we'll use threading events and locks to synchronize the execution of the algorithm (making each client, when sending a message, wait to have received the messages of all its neighbors before sending the next message)
- we'll add two methods, `_from_message` and `_to_message` to convert between the message exchange formats (which will need to be overridden in subclasses, based on the algorithm)

The main message exchange will be done in the `_exchange_values` function.

```python
import threading
from abc import abstractmethod
from collections import defaultdict

from nvflare.apis.dxo import DXO, DataKind
from nvflare.apis.event_type import EventType
from nvflare.apis.signal import Signal


class SyncAlgorithmExecutor(Executor):
    def __init__(self):
        super().__init__()
        ... # other attributes

        self.neighbors_values = defaultdict(dict)

        self.sync_waiter = threading.Event()
        self.lock = threading.Lock()


    def _exchange_values(self, fl_ctx: FLContext, value: any, iteration: int):
        engine = fl_ctx.get_engine()

        # Clear the event before starting the exchange
        self.sync_waiter.clear()

        # Send message to neighbors
        _ = engine.send_aux_request(
            targets=[neighbor.id for neighbor in self.neighbors],
            topic="send_value",
            request=DXO(
                data_kind=DataKind.METRICS,
                data={
                    "value": self._to_message(value),
                    "iteration": iteration,
                },
            ).to_shareable(),
            timeout=10,
            fl_ctx=fl_ctx,
        )

        # check if all neighbors sent their values
        if len(self.neighbors_values[iteration]) < len(self.neighbors):
            # if not, wait for them, max 10 seconds
            if not self.sync_waiter.wait(timeout=10):
                self.system_panic("failed to receive values from all neighbors", fl_ctx)
                return

    def _handle_neighbor_value(
        self, topic: str, request: Shareable, fl_ctx: FLContext
    ) -> Shareable:
        sender = request.get_peer_props()["__identity_name__"]
        data = from_shareable(request).data
        iteration = data["iteration"]

        with self.lock:
            self.neighbors_values[iteration][sender] = self._from_message(data["value"])
            # Check if all neighbor values have been received
            if len(self.neighbors_values[iteration]) >= len(self.neighbors):
                self.sync_waiter.set()  # Signal that we have all neighbor values
        return make_reply(ReturnCode.OK)

    def handle_event(self, event_type: str, fl_ctx: FLContext):
        if event_type == EventType.START_RUN:
            engine = fl_ctx.get_engine()

            engine.register_aux_message_handler(
                topic="send_value", message_handle_func=self._handle_neighbor_value
            )

    def _to_message(self, x):
        return x

    def _from_message(self, x):
        return x

```

That's it for the synchronous message exchange. Notice that `neighbors_values` needs to maintain a dictionary of received values per iteration. 
This is because, different parts of a network may be at different iterations of the algorithm (plus or minus 1 at most) - this means that I could receive a message from a neighbor valid for iteration `t+1` when I'm still at iteration `t`. Since that message won't be sent again, I need to store it. To avoid the `neighbors_values` to grow indefinitely, we'll delete its content at iteration `t` after having consumed its values and moving to the next iteration in the algorithm. We'll see that in the next section.

Moving forward, now that we have a way to store the config and exchange messages with the neighbors, we can move on to implementing the algorithmic part. For this base `SyncAlgorithmExecutor`, we'll just implement the main logic in the `execute` method, based on an abstract `run_algorithm` to be overridden by each specific algorithm.

```python
class SyncAlgorithmExecutor(Executor):
    
    ...
    
    def execute(
        self,
        task_name: str,
        shareable: Shareable,
        fl_ctx: FLContext,
        abort_signal: Signal,
    ):
        if task_name == "config":
            # Receive topology from the server
            self._load_config(shareable, fl_ctx)
            return make_reply(ReturnCode.OK)

        elif task_name == "run_algorithm":
            self.run_algorithm(fl_ctx, shareable, abort_signal)
            return make_reply(ReturnCode.OK)
        else:
            self.log_warning(fl_ctx, f"Unknown task name: {task_name}")
            return make_reply(ReturnCode.TASK_UNKNOWN)

    @abstractmethod
    def run_algorithm(
        self, fl_ctx: FLContext, shareable: Shareable, abort_signal: Signal
    ):
        """Executes the algorithm"""
        pass
    
    ...
```

And that's all. The full implementation is in `nvflare/app_opt/p2p/executors/sync_executor.py` - note that the implementation of the `SyncAlgorithmExecutor` in `nvflare.app_opt.p2p` is a subclass of `BaseDistOptExecutor`, defined in `nvflare/app_opt/p2p/executors/base_dist_opt_executor.py`. It contains a few additional attributes (namely `self.id` and `self.client_name`) to identify the client, which are potentially useful in algorithms, and two additional methods `_pre_algorithm_run` and `_post_algorithm_run` to be overridden by each specific algorithm to execute some code before and after the algorithm execution, respectively.

### An example: the `ConsensusExecutor`

Now that we have built all the main foundations, we can easily implement any custom P2P algorithm. For example, let's implement a slightly simplified version of the `ConsensusExecutor` that will be used in the next section and whose full implementation is in `nvflare/app_opt/p2p/executors/consensus.py`.

```python
import torch

class ConsensusExecutor(SyncAlgorithmExecutor):

    def __init__(
        self,
        initial_value: float | None = None,
    ):
        super().__init__()
        if initial_value is None:
            initial_value = random.random()
        self.current_value = initial_value
        self.value_history = [self.current_value]

    def run_algorithm(self, fl_ctx, shareable, abort_signal):
        iterations = from_shareable(shareable).data["iterations"]

        for iteration in range(iterations):
            # 1. exchange values
            self._exchange_values(
                fl_ctx, value=self.current_value, iteration=iteration
            )

            # 2. compute new value
            current_value = self.current_value * self._weight
            for neighbor in self.neighbors:
                current_value += (
                    self.neighbors_values[iteration][neighbor.id] * neighbor.weight
                )

            # 3. store current value
            self.current_value = current_value

            # free memory that's no longer needed
            del self.neighbors_values[iteration]

```

As you can see, it's basically just a matter of implementing the algorithm logic in the `run_algorithm` method. Feel free to explore the `nvflare.app_opt.p2p` module to see how other algorithms are implemented.